# Training

In [4]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310'

In [22]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


## Build

In [20]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


In [46]:
!docker build -t 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2 -f "/home/ec2-user/SageMaker/esame_devops_rogic/docker/Dockerfile.train" /home/ec2-user/SageMaker/esame_devops_rogic

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 278B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.2s (9/9) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 278B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [29]:
!cd ..; ls
print("########################################################")
!cd .; ls

app  data  docker  module  notebook  README.md
########################################################
main.ipynb


## Esecuzione Locale

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os
import pandas as pd
role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!SM_MODEL_DIR="./data/output" python main.py

In [ ]:
!tar -xzf ./data/output/model.tar.gz

## Init repo on ECR

In [24]:
%%sh
# Specify an image name
image_name=my-tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pippo-2"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"


image_name: my-tensorflow-training ######################
account: 591530427655 ######################
region: us-east-1 ######################
fullname: 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2 ######################
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:591530427655:repository/my-tensorflow-training",
        "registryId": "591530427655",
        "repositoryName": "my-tensorflow-training",
        "repositoryUri": "591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training",
        "createdAt": 1732266683.696,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [25]:
!docker images

REPOSITORY                                                            TAG                        IMAGE ID       CREATED         SIZE
591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training   2.12.0-cpu-py310-pippo-2   7e1ae99f425d   3 minutes ago   8.59GB


In [ ]:
#!docker rmi tensorflow-training:2.12.0-cpu-py310-pippo-2

In [ ]:
#!docker tag tensorflow-training:2.12.0-cpu-py310-pippo-2 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2

In [ ]:
#!docker tag bec0922c4a51 tensorflow-training:2.12.0-cpu-py310-pippo-2

## Push image

In [ ]:
#!aws sagemaker describe-notebook-instance \
    #--notebook-instance-name CFNNotebookInstance-fBxJOmvnjTpG \
    #--region us-east-1

In [ ]:
#!aws iam attach-role-policy \
    #--role-name c133245a3375001l7613526t1w00-SageMakerExecutionRole-XxDt97bVnO4D \
    #--policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

In [47]:
!docker push 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2

The push refers to repository [591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training]

85be2949: Preparing 
85ba51ae: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
0445c449: Layer already exists 2.12.0-cpu-py310-pippo-2: digest: sha256:266465515f6a6dc772ca27e2a0d43f53b4080c17a6ca86080dcb2655446c9349 size: 7045


## Esecuzione su ECR

In [48]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os
import pandas as pd

role=get_execution_role()

MODEL_INPUT_PATH = 's3://marcorogicesame/data/input/user_behavior_dataset.csv'

MODEL_OUTPUT_PATH = 's3://marcorogicesame/data/output'

INSTANCE_TYPE = 'ml.p2.xlarge'

hyperparameters={'epochs': 50}

estimator=Estimator(
    image_uri='591530427655.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:2.12.0-cpu-py310-pippo-2',
    role=role,
    instance_count=1,
    instance_type=INSTANCE_TYPE,
    hyperparameters=hyperparameters,
    output_path=MODEL_OUTPUT_PATH
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit(MODEL_INPUT_PATH)
print('##### ESTIMATOR FIT COMPLETED')

INFO:sagemaker:Creating training-job with name: my-tensorflow-training-2024-11-22-10-14-44-810


##### ESTIMATOR FIT STARTED
2024-11-22 10:14:46 Starting - Starting the training job...
2024-11-22 10:15:11 Starting - Preparing the instances for training...
2024-11-22 10:15:50 Downloading - Downloading input data...
2024-11-22 10:16:10 Downloading - Downloading the training image...............
2024-11-22 10:18:37 Training - Training image download completed. Training in progress....2024-11-22 10:19:10.003876: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 10:19:10.007770: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-22 10:19:10.019669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732270750.039213      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to r

UnexpectedStatusException: Error for Training job my-tensorflow-training-2024-11-22-10-14-44-810: Failed. Reason: AlgorithmError: NotFoundError:
ExitCode 1
ErrorMessage "FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/input/data/input/user_behavior_dataset.csv'"
Command "/usr/local/bin/python3.10 train.py --epochs 50", exit code: 1. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

# Build immagine inferenza

In [ ]:
!docker build -t my-flask-inference -f "/home/ec2-user/SageMaker/esame_devops_rogic/docker/Dockerfile.inference" /home/ec2-user/SageMaker/esame_devops_rogic

In [ ]:
!docker images

# Init ECR per immagine inferenza

In [ ]:
%%sh
# Specify an image name
image_name=my-flask-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:1.12.0-cpu"

echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 591530427655.dkr.ecr.us-east-1.amazonaws.com

# Push dell'immagine di inferenza all'interno dell'ECR

In [ ]:
!docker tag my-flask-inference:latest 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest

In [ ]:
!docker push 591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest

# Prova inferenza in locale

In [ ]:
from keras import models
import numpy as np
import json

print(os.path.exists(f"/home/ec2-user/SageMaker/esame_devops_rogic/model.keras"))
model = models.load_model("/home/ec2-user/SageMaker/esame_devops_rogic/model.keras")
type(model)

predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

In [ ]:
!docker images

# Creazione endpoint e deploy

In [ ]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'il-top'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '591530427655.dkr.ecr.us-east-1.amazonaws.com/my-flask-inference:latest', ##Inference URI in ECR
        'ModelDataUrl': 's3://brunobucket69/data/output/my-tensorflow-training-2024-11-21-20-56-58-105/output/model.tar.gz',
    })


In [ ]:
endpoint_config_name = 'boiadee'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1", 
            "ModelName": model_name,
            "InstanceType": instance_type, 
            "InitialInstanceCount": 1 
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

endpoint_name = 'primo-endpoint-boia'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

In [ ]:
import boto3
import json

runtime_client = boto3.client('sagemaker-runtime', region_name=aws_region)

payload = json.dumps({"instances": [[0.475, 0.37, 0.125, 0.5095, 0.2165, 0.1125, 0.165, 9]]})

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  
    Body=payload                     
)

result = json.loads(response['Body'].read().decode())
print(f"Inference result: {result}")
